In [599]:
import math 
import time 
import tkinter as tk
import matplotlib.pyplot as plt

In [600]:
class SensorBase:
    def __init__(self, name):
        self.name = name
        self.value = None

    # def update(self, kwargs):
    #     raise NotImplementedError("Subclasses should implement this!")
    
    def read(self):
        return self.value 



In [601]:
class Pump_hcl:
    def __init__(self, name="pump", hcl_max_flowrate=0.00000069, transition_time=5):    
        self.name = name
        self.flowrate = 0.0                          # Current outlet flowrate (L/s)
        self.max_flowrate = hcl_max_flowrate            # Max flowrate when fully on
        self.transition_time = transition_time      # Time to reach full flow (s)

        self.state = "off"                           # Pump states: off, transitioning_on, on, transitioning_off
        self.transition_progress = 0.0               # Between 0 (off) and 1 (fully on)

    def turn_on(self):
        if self.state == "off":
            self.state = "transitioning_on"
            self.transition_progress = 0.0

    def turn_off(self):
        if self.state == "on":
            self.state = "transitioning_off"
            self.transition_progress = 1.0

    def update(self, dt):
        if self.state == "transitioning_on":
            self.transition_progress += dt / self.transition_time
            if self.transition_progress >= 1.0:
                self.transition_progress = 1.0
                self.state = "on"

        elif self.state == "transitioning_off":
            self.transition_progress -= dt / self.transition_time
            if self.transition_progress <= 0.0:
                self.transition_progress = 0.0
                self.state = "off"

    def get_outlet_flowrate(self):
        # No need to multiply by dt here; return flowrate in L/s
        self.flowrate = self.transition_progress * self.max_flowrate
        return self.flowrate


In [602]:
class ph_sensor(SensorBase):

    def __init__(self, initial_ph = 7.5):
        super().__init__("pH Sensor")
        self.value = initial_ph

    def update(self, flowrate_water=0.7, ph_water =7.5, hcl_pump_on = False, hcl_flow = 0.0, pH_hcl = 1.0, max_flow = 0.0000025):
        # Update pH based on water flow and HCl pump rate
        # hcl_flow = Pump_hcl.get_outlet_flowrate(self)
        initial_conc = 10**(-ph_water)
        hcl_conc = 10**(-pH_hcl)
        if hcl_flow > 0.0:
            # Calculate the amount of HCl added to the water
            hcl_added = hcl_flow * hcl_conc

            # Calculate the new concentration of H+ ions in the water
            new_conc = initial_conc + hcl_added

            # Calculate the new pH value
            new_ph = -math.log10(new_conc)


        else:

            # If the pump is off, the pH remains the same
            new_ph = ph_water


        # Update the sensor value
        self.value = new_ph

In [603]:
class hcl_PumpGUI:
    def __init__(self,parent, pump):
        self.pump = pump

        self.frame = tk.LabelFrame(parent, text="HCL Pump Control", padx=10, pady=10)
        self.frame.pack(side=tk.LEFT, padx=10, pady=10)

        self.status_label = tk.Label(self.frame, text="Status: ---", font=("Arial", 14))
        self.status_label.pack(pady=5)

        self.toggel_button = tk.Button(
            self.frame,
            text = "Toggle Pump",
            command = self.toggle_pump,
            font = ("Arial", 14)
        )
        self.toggel_button.pack(pady=5)

        self.update_gui()

    def toggle_pump(self):
        if self.pump.state in  ["off", "transitioning_off"]: ## if the pump is off or transitioning off, turn it on
            self.pump.turn_on()
        elif self.pump.state in ["on", "transitioning_on"]:
            self.pump.turn_off()

    def update_gui(self):
        if self.pump.state == 'on':
            text = "Status: ON (100%)"
            color = "green"
        elif self.pump.state == 'off':
            text = "Status: OFF (0%)"
            color = "red"
        elif self.pump.state == 'transitioning_on':
            percent = self.pump.transition_progress * 100
            text = f"Transitioning ON ({percent:.0f}%)"
            color = "orange"
        elif self.pump.state == 'transitioning_off':
            percent = self.pump.transition_progress * 100
            text = f"Transitioning OFF ({percent:.0f}%)"
            color = "yellow"
        else:
            text = "Status: UNKNOWN"
            color = "black"

        self.status_label.config(text=text, fg=color)
        self.frame.after(1000, self.update_gui)


In [604]:
# class hcl_SimulationGUI:
#     def __init__(self, root):
#         self.root = root
#         self.root.title("Water Treatment Simulation")

#         # System Components
#         self.pump = Pump_hcl()
#         self.sensor = ph_sensor()

#         # Time tracking
#         self.time = 0
#         self.dt = 1  # 1 second timestep

#         # GUI sections
#         self.left_panel = tk.Frame(root)
#         self.left_panel.pack(side=tk.LEFT)

#         self.right_panel = tk.Frame(root)
#         self.right_panel.pack(side=tk.RIGHT)

#         # Pump GUI
#         self.pump_gui = PumpGUI(self.left_panel, self.pump)

#         # Info Labels
#         self.time_label = tk.Label(self.right_panel, text="Time: 0s", font=("Arial", 14))
#         self.flow_label = tk.Label(self.right_panel, text="Flowrate: 0.00e+00 L/s", font=("Arial", 14))
#         self.ph_label = tk.Label(self.right_panel, text="pH: 7.5000", font=("Arial", 14))

#         self.time_label.pack(pady=5)
#         self.flow_label.pack(pady=5)
#         self.ph_label.pack(pady=5)

#         self.run_simulation()


In [605]:
# class SimulationGUI:
#     def __init__(self, root):
#         self.root = root
#         self.root.title("Water Treatment Simulation")

#         # System Components
#         self.pump = Pump_hcl()
#         self.sensor = ph_sensor()

#         # Time tracking
#         self.time = 0
#         self.dt = 1  # 1 second timestep

#         # GUI sections
#         self.left_panel = tk.Frame(root)
#         self.left_panel.pack(side=tk.LEFT)

#         self.right_panel = tk.Frame(root)
#         self.right_panel.pack(side=tk.RIGHT)

#         # Pump GUI
#         self.pump_gui = PumpGUI(self.left_panel, self.pump)

#         # Info Labels
#         self.time_label = tk.Label(self.right_panel, text="Time: 0s", font=("Arial", 14))
#         self.flow_label = tk.Label(self.right_panel, text="Flowrate: 0.00e+00 L/s", font=("Arial", 14))
#         self.ph_label = tk.Label(self.right_panel, text="pH: 7.5000", font=("Arial", 14))

#         self.time_label.pack(pady=5)
#         self.flow_label.pack(pady=5)
#         self.ph_label.pack(pady=5)

#         self.run_simulation()

#     def run_simulation(self):
#         self.pump.update(self.dt)
#         flow = self.pump.get_outlet_flowrate()
#         self.sensor.update(hcl_flow=flow)

#         # Update right panel info
#         self.time_label.config(text=f"Time: {self.time}s")
#         self.flow_label.config(text=f"Flowrate: {flow:.2e} L/s")
#         self.ph_label.config(text=f"pH: {self.sensor.read():.4f}")

#         self.time += 1
#         self.root.after(1000, self.run_simulation)

# if __name__ == "__main__":
#     root = tk.Tk()
#     app = SimulationGUI(root)
#     root.mainloop()






In [606]:
class Pump_naocl:
    def __init__(self, name="pump_naocl", naocl_max_flowrate=0.00000069, transition_time=5):    
        self.name = name
        self.flowrate = 0.0                          # Current outlet flowrate (L/s)
        self.max_flowrate = naocl_max_flowrate          # Max flowrate when fully on
        self.transition_time = transition_time      # Time to reach full flow (s)

        self.state = "off"                           # Pump states: off, transitioning_on, on, transitioning_off
        self.transition_progress = 0.0               # Between 0 (off) and 1 (fully on)

    def turn_on(self):
        if self.state == "off":
            self.state = "transitioning_on"
            self.transition_progress = 0.0

    def turn_off(self):
        if self.state == "on":
            self.state = "transitioning_off"
            self.transition_progress = 1.0

    def update(self, dt):
        if self.state == "transitioning_on":
            self.transition_progress += dt / self.transition_time
            if self.transition_progress >= 1.0:
                self.transition_progress = 1.0
                self.state = "on"

        elif self.state == "transitioning_off":
            self.transition_progress -= dt / self.transition_time
            if self.transition_progress <= 0.0:
                self.transition_progress = 0.0
                self.state = "off"

    def get_outlet_flowrate(self):
        # No need to multiply by dt here; return flowrate in L/s
        self.flowrate = self.transition_progress * self.max_flowrate
        return self.flowrate

In [607]:
class orp_sensor(SensorBase):
    def __init__(self, initial_orp=700, initial_ph=7.5, initial_hocl=1.626):
        super().__init__("ORP Sensor")
        self.orp = initial_orp
        self.ph = initial_ph
        self.hocl = initial_hocl  # ✅ track total HOCl over time

    def update(self, flowrate_water=0.7, naocl_flow=0.0, naocl_ppm=2.0):
        # Update HOCl concentration based on NaOCl flow
        if naocl_flow > 0.0:
            delta_hocl = naocl_flow * naocl_ppm  # ✅ added HOCl in ppm equivalent
            self.hocl = min(self.hocl + (naocl_flow*naocl_ppm), 2.8 )  # cap to realistic level

        # ORP formula
        try:
            # self.orp = 2025.333 - 158.667 * float(self.ph) - 86.667 * float(self.hocl)

            self.orp = 78.55 * float(self.hocl) + 582.78
        except Exception as e:
            print("[ORP ERROR]", e)
            self.orp = 600  # fallback safe ORP

    def read(self):
        return self.orp

    def reset(self, new_ph=7.5, new_hocl=2.72):
        self.ph = new_ph
        self.hocl = new_hocl


In [608]:
class NaOCl_PumpGUI:
    def __init__(self,parent, pump):
        self.pump = pump

        self.frame = tk.LabelFrame(parent, text=" NaOCl Pump Control", padx=10, pady=10)
        self.frame.pack(side=tk.LEFT, padx=10, pady=10)

        self.status_label = tk.Label(self.frame, text="Status: ---", font=("Arial", 14))
        self.status_label.pack(pady=5)

        self.toggel_button = tk.Button(
            self.frame,
            text = "Toggle Pump",
            command = self.toggle_pump,
            font = ("Arial", 14)
        )
        self.toggel_button.pack(pady=5)

        self.update_gui()

    def toggle_pump(self):
        if self.pump.state in  ["off", "transitioning_off"]: ## if the pump is off or transitioning off, turn it on
            self.pump.turn_on()
        elif self.pump.state in ["on", "transitioning_on"]:
            self.pump.turn_off()

    def update_gui(self):
        if self.pump.state == 'on':
            text = "Status: ON (100%)"
            color = "green"
        elif self.pump.state == 'off':
            text = "Status: OFF (0%)"
            color = "red"
        elif self.pump.state == 'transitioning_on':
            percent = self.pump.transition_progress * 100
            text = f"Transitioning ON ({percent:.0f}%)"
            color = "orange"
        elif self.pump.state == 'transitioning_off':
            percent = self.pump.transition_progress * 100
            text = f"Transitioning OFF ({percent:.0f}%)"
            color = "yellow"
        else:
            text = "Status: UNKNOWN"
            color = "black"

        self.status_label.config(text=text, fg=color)
        self.frame.after(1000, self.update_gui)

In [609]:
class conductivity_sensor(SensorBase):
    def __init__(self, initial_value=200.0):
        super().__init__("Conductivity Sensor")
        self.value = initial_value  # in µS/cm

    def update(self):
        # No change — fixed raw water reading for now
        pass

    def read(self):
        return self.value


In [610]:
class Pump_nacl:
    def __init__(self, name="pump_naocl", naocl_max_flowrate=0.00000069, transition_time=5):    
        self.name = name
        self.flowrate = 0.0                          # Current outlet flowrate (L/s)
        self.max_flowrate = naocl_max_flowrate          # Max flowrate when fully on
        self.transition_time = transition_time      # Time to reach full flow (s)

        self.state = "off"                           # Pump states: off, transitioning_on, on, transitioning_off
        self.transition_progress = 0.0               # Between 0 (off) and 1 (fully on)

    def turn_on(self):
        if self.state == "off":
            self.state = "transitioning_on"
            self.transition_progress = 0.0

    def turn_off(self):
        if self.state == "on":
            self.state = "transitioning_off"
            self.transition_progress = 1.0

    def update(self, dt):
        if self.state == "transitioning_on":
            self.transition_progress += dt / self.transition_time
            if self.transition_progress >= 1.0:
                self.transition_progress = 1.0
                self.state = "on"

        elif self.state == "transitioning_off":
            self.transition_progress -= dt / self.transition_time
            if self.transition_progress <= 0.0:
                self.transition_progress = 0.0
                self.state = "off"

    def get_outlet_flowrate(self):
        # No need to multiply by dt here; return flowrate in L/s
        self.flowrate = self.transition_progress * self.max_flowrate
        return self.flowrate
    

In [611]:
class NaCl_PumpGUI:
    def __init__(self,parent, pump):
        self.pump = pump

        self.frame = tk.LabelFrame(parent, text=" NaCl Pump Control", padx=10, pady=10)
        self.frame.pack(side=tk.LEFT, padx=10, pady=10)

        self.status_label = tk.Label(self.frame, text="Status: ---", font=("Arial", 14))
        self.status_label.pack(pady=5)

        self.toggel_button = tk.Button(
            self.frame,
            text = "Toggle Pump",
            command = self.toggle_pump,
            font = ("Arial", 14)
        )
        self.toggel_button.pack(pady=5)

        self.update_gui()

    def toggle_pump(self):
        if self.pump.state in  ["off", "transitioning_off"]: ## if the pump is off or transitioning off, turn it on
            self.pump.turn_on()
        elif self.pump.state in ["on", "transitioning_on"]:
            self.pump.turn_off()

    def update_gui(self):
        if self.pump.state == 'on':
            text = "Status: ON (100%)"
            color = "green"
        elif self.pump.state == 'off':
            text = "Status: OFF (0%)"
            color = "red"
        elif self.pump.state == 'transitioning_on':
            percent = self.pump.transition_progress * 100
            text = f"Transitioning ON ({percent:.0f}%)"
            color = "orange"
        elif self.pump.state == 'transitioning_off':
            percent = self.pump.transition_progress * 100
            text = f"Transitioning OFF ({percent:.0f}%)"
            color = "yellow"
        else:
            text = "Status: UNKNOWN"
            color = "black"

        self.status_label.config(text=text, fg=color)
        self.frame.after(1000, self.update_gui)

In [612]:
class SimulationGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Water Treatment Simulation")

        # System Components
        self.pump = Pump_hcl()
        self.sensor = ph_sensor()
        self.sensor2 = orp_sensor()
        self.pump2 = Pump_naocl()
        self.sensor3 = conductivity_sensor()
        self.pump3 = Pump_nacl()

        # Time tracking
        self.time = 0
        self.dt = 1  # 1 second timestep

        # Create a right panel for grouping labels
        self.right_panel = tk.Frame(self.root)
        self.right_panel.pack(side=tk.RIGHT, padx=10, pady=10)

        # Create a sub-frame inside right_panel to group labels nicely
        self.info_frame = tk.Frame(self.right_panel)
        self.info_frame.pack(padx=10, pady=10)

        # Time Label
        self.time_label = tk.Label(self.info_frame, text="Time: 0s", font=("Arial", 14))
        self.time_label.grid(row=0, column=0, columnspan=2, sticky='w')

        # HCl Info
        self.hcl_flow_label = tk.Label(self.info_frame, text="HCl Flowrate: 0.00e+00 L/s", font=("Arial", 14))
        self.ph_label = tk.Label(self.info_frame, text="pH: 7.5000", font=("Arial", 14))
        self.hcl_flow_label.grid(row=1, column=0, sticky='w')
        self.ph_label.grid(row=1, column=1, sticky='w')

        # NaOCl Info
        self.naocl_flow_label = tk.Label(self.info_frame, text="NaOCl Flowrate: 0.00e+00 L/s", font=("Arial", 14))
        self.ppm_label = tk.Label(self.info_frame, text="HOCl (ppm): 0.00", font=("Arial", 14))
        self.naocl_flow_label.grid(row=2, column=0, sticky='w')
        self.ppm_label.grid(row=2, column=1, sticky='w')

        # ORP Info
        self.orp_label = tk.Label(self.info_frame, text="ORP: ---- mV", font=("Arial", 14, "bold"))
        self.orp_label.grid(row=3, column=0, columnspan=2, sticky='w')

        self.left_panel = tk.Frame(self.root)
        self.left_panel.pack(side=tk.LEFT, padx=10, pady=10)

        self.pump_gui = hcl_PumpGUI(self.left_panel, self.pump)
        self.naocl_gui = NaOCl_PumpGUI(self.left_panel, self.pump2)
        self.nacl_gui = NaCl_PumpGUI(self.left_panel, self.pump3)

        #nacl info 
        self.naclflow_label = tk.Label(self.info_frame, text="NaCl Flowrate: 0.00e+00 L/s", font=("Arial", 14))
        self.naclflow_label.grid(row=4, column=0, sticky='w')
        self.conductivity_label = tk.Label(self.info_frame, text="Conductivity: 0.00 µS/cm", font=("Arial", 14))
        self.conductivity_label.grid(row=4, column=1, sticky='w')

        

        # Remove the pack() calls since grid() is used for layout
        pass


        self.run_simulation()

    def run_simulation(self):
        self.pump.update(self.dt)
        hcl_flow = self.pump.get_outlet_flowrate()
        self.sensor.update(hcl_flow=hcl_flow)

        self.pump2.update(self.dt)
        naocl_flow = self.pump2.get_outlet_flowrate()
        ph = float(self.sensor.read()) # Get the current pH from the sensor
        # print(f"[DEBUG] ORP calc → HOCl: {self.sensor2.hocl} ({type(self.sensor2.hocl)}), pH: {ph} ({type(ph)})")


        self.sensor2.update(ph,naocl_flow=naocl_flow)

        # Update the conductivity sensor 
        # self.sensor3 = conductivity_sensor()
        self.sensor3.update()

        # Update the NaCl pump
        self.pump3.update(self.dt)
        nacl_flow = self.pump3.get_outlet_flowrate()

        # Update right panel info
        self.time_label.config(text=f"Time: {self.time}s")
        self.hcl_flow_label.config(text=f"HCl Flowrate: {hcl_flow:.7e} L/s")
        self.ph_label.config(text=f"pH: {self.sensor.read():.4f}")
        self.naocl_flow_label.config(text=f"NaOCl Flowrate: {naocl_flow:.7e} L/s")
        self.orp_label.config(text=f"ORP: {self.sensor2.read():.4f} mV")
        self.conductivity_label.config(text=f"Conductivity: {self.sensor3.read():.2f} µS/cm")
        self.naclflow_label.config(text=f"NaCl Flowrate: {self.pump3.get_outlet_flowrate():.7e} L/s")
        # print(f"[DEBUG] HOCl in sensor2: {self.sensor2.hocl}")


        self.time += 1
        self.root.after(1000, self.run_simulation)

if __name__ == "__main__":
    root = tk.Tk()
    app = SimulationGUI(root)
    root.mainloop()